# Visualisation of LFP & Sleep scoring

### Load LFP and packages

In [1]:
from scipy import signal
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.widgets import Slider, Button, Cursor
from scipy import fftpack
import pandas as pd
from pathlib import Path
import os
from IPython.display import display
from ipyfilechooser import FileChooser
from ephyviewer import mkQApp, MainViewer, TraceViewer
from ephyviewer import AnalogSignalSourceWithScatter
import ephyviewer
from scipy.stats import zscore
from scipy.interpolate import interp1d
from itertools import groupby
from ephyviewer import mkQApp, MainViewer, TraceViewer, TimeFreqViewer
from ephyviewer import InMemoryAnalogSignalSource
import ephyviewer

from scipy.signal import find_peaks
from scipy.signal import chirp, find_peaks, peak_widths

#%matplotlib widget

In [2]:
dpath = "//10.69.168.1/crnldata/forgetting/Carla/"

fc1 = FileChooser(dpath,select_default=True, show_only_dirs = False, title = "<b>Open Ephys folder</b>")
display(fc1)

# Sample callback function
def update_my_folder(chooser):
    global dpath
    dpath = chooser.selected
    %store dpath
    return 

# Register callback function
fc1.register_callback(update_my_folder)

FileChooser(path='\\10.69.168.1\crnldata\forgetting\Carla', filename='', title='<b>Open Ephys folder</b>', sho…

Stored 'dpath' (str)


In [3]:
folder_base = Path(dpath) 
DataRec = np.fromfile(folder_base, dtype="int16")
All = DataRec.reshape(-1,96)
All.shape
#DataRec2 = np.fromfile(folder_base, dtype="uint16")
#All2 = DataRec2.reshape(-1,32) 
#All2.shape

Channels = '//10.69.168.1/crnldata/forgetting/Carla/OpenEphysData/LFPChannels_perMice.xlsx' 


mice=folder_base.parts[-8].split('_')[0]

mice='Vert'

print(mice)

allchannels = pd.read_excel(Channels)


PFCch1=int(allchannels[mice][0].split('_')[0])
PFCch2=int(allchannels[mice][0].split('_')[1])
CA1ch1=int(allchannels[mice][2].split('_')[0])
CA1ch2=int(allchannels[mice][2].split('_')[1])
S1ch1=int(allchannels[mice][1].split('_')[0])
S1ch2=int(allchannels[mice][1].split('_')[1])
EMGch1=int(allchannels[mice][3])
oPFCch1=int(allchannels[mice][4].split('_')[0])
oPFCch2=int(allchannels[mice][4].split('_')[1])
EnthCch1=int(allchannels[mice][5].split('_')[0])
EnthCch2=int(allchannels[mice][5].split('_')[1])
RHch1=int(allchannels[mice][6].split('_')[0])
RHch2=int(allchannels[mice][6].split('_')[1])


Vert


In [5]:
#Channels 0-31
PFC  =  All[:, (PFCch1)]-All[:, (PFCch2)] 
CA1  =  All[:, (CA1ch1)]-All[:, (CA1ch2)]
#CA1  =  All[:, (CA1ch1)] ou CA1  =  All[:, (CA1ch2)] si problème avec une électode
S1  =  All[:, (S1ch1)]-All[:, (S1ch2)] 
EMG  =  All[:, (EMGch1)]
#oPFC  =  All[:, (oPFCch1)]-All[:, (oPFCch2)] 
oPFC  =  All[:, (oPFCch1)]
EnthC  =  All[:, (EnthCch1)]-All[:, (EnthCch2)] 
RH  =  All[:, (RHch1)]-All[:, (RHch2)] 


combined = np.stack([zscore(S1),zscore(PFC), zscore(CA1), zscore(EMG), zscore(oPFC), zscore(EnthC), zscore(RH)], axis = 1)

In [9]:
allchannels

,MICE,Vert test,Jaune test,Rouge,Bleu,Violet,Jaune,Vert,Orange
0,PFC,29.31,"28,30",NaN,NaN,13.15,17.19,5.70,5.70
1,S1,"1,3","0,2",5.70,27.29,17.19,5.70,13.15,13.15
2,CA1,"13,17","14,18",27.29,17.19,1.30,9.11,27.29,27.29
3,EMG,9,9,22.00,22.00,22.00,22.00,22.00,22.00
4,oPFC,"5,7","5,7",9.11,1.30,9.11,27.29,9.11,9.11
5,EnthC,21.23,"21,23",1.30,9.11,27.29,1.30,17.19,17.19
6,RH,NaN,NaN,17.19,5.70,5.70,13.15,1.30,1.30


In [5]:
#Channels 32-63
#PFC  =  All[:, (PFCch1+32)]-All[:, (PFCch2+32)] 
CA1  =  All[:, (CA1ch1+32)]-All[:, (CA1ch2+32)] 
S1  =  All[:, (S1ch1+32)]-All[:, (S1ch2+32)] 
EMG  =  All[:, (EMGch1+32)]
oPFC  =  All[:, (oPFCch1+32)]-All[:, (oPFCch2+32)] 
EnthC  =  All[:, (EnthCch1+32)]-All[:, (EnthCch2+32)] 
RH  =  All[:, (RHch1+32)]-All[:, (RHch2+32)] 


#combined = np.stack([zscore(S1),zscore(PFC), zscore(CA1), zscore(EMG), zscore(oPFC), zscore(EnthC), zscore(RH)], axis = 1)

combined = np.stack([zscore(S1), zscore(CA1), zscore(EMG), zscore(oPFC), zscore(EnthC),zscore(RH)], axis = 1)

In [5]:
#Channels 64-95
PFC  =  All[:, (PFCch1+64)]-All[:, (PFCch2+64)] 
CA1  =  All[:, (CA1ch1+64)]-All[:, (CA1ch2+64)] 
S1  =  All[:, (S1ch1+64)]-All[:, (S1ch2+64)] 
EMG  =  All[:, (EMGch1+64)]
oPFC  =  All[:, (oPFCch1+64)]-All[:, (oPFCch2+64)] 
EnthC  =  All[:, (EnthCch1+64)]-All[:, (EnthCch2+64)] 
RH  =  All[:, (RHch1+64)]-All[:, (RHch2+64)] 

combined = np.stack([zscore(S1),zscore(PFC), zscore(CA1), zscore(EMG), zscore(oPFC), zscore(EnthC), zscore(RH)], axis = 1)

# EphyViewer


In [7]:
from ephyviewer import mkQApp, MainViewer, TraceViewer, TimeFreqViewer, CsvEpochSource, EpochEncoder,EpochViewer
from ephyviewer import InMemoryAnalogSignalSource
from ephyviewer import InMemorySpikeSource

app = mkQApp()
win = MainViewer(debug=True, show_auto_scale=True)

# LFPs

sample_rate = 1000
t_start = 0.

#source = AnalogSignalSourceWithScatter(combined, sample_rate, t_start, scatter_indexes, scatter_channels, scatter_colors= {0: '#FFFFFF', 1: '#222222', 2: '#FFFFFF', 3: '#222222', 4: '#FFFFFF', 5: '#222222'}, channel_names=['S1','PFC', 'CA1', 'EMG', 'Scoring'])
#source =InMemoryAnalogSignalSource(combined, sample_rate, t_start, channel_names=['S1', 'CA1', 'EMG','oPFC', 'EnthC', 'RH'])
source =InMemoryAnalogSignalSource(combined, sample_rate, t_start, channel_names=['S1','PFC', 'CA1', 'EMG','oPFC', 'EnthC', 'RH'])
view1 = TraceViewer(source=source)

view1.params['display_labels'] = True
view1.params['scale_mode'] = 'same_for_all'
view1.auto_scale()
view1.by_channel_params['ch0', 'color'] = '#FF88FF' #FF0000 red, #00FF00 green, and #0000FF blue
view1.by_channel_params['ch1', 'color'] = '#88FF88'
view1.by_channel_params['ch2', 'color'] = '#8888FF'
view1.by_channel_params['ch3', 'color'] = '#FFFFFF'
view1.by_channel_params['ch4', 'color'] = '#FF8877'
view1.by_channel_params['ch5', 'color'] = '#777777'
view1.by_channel_params['ch6', 'color'] = '#FFFF99'


# FFT
view3 = TimeFreqViewer(source=source, name='FFT')

view3.params['show_axis'] = True
view3.params['timefreq', 'f_start'] = 1
view3.params['timefreq', 'f_stop'] = 50
view3.params['timefreq', 'deltafreq'] = 1 #interval in Hz

view3.by_channel_params['ch0', 'clim'] = 1
view3.by_channel_params['ch1', 'clim'] = 1
view3.by_channel_params['ch2', 'clim'] = 1
view3.by_channel_params['ch4', 'clim'] = 1
view3.by_channel_params['ch5', 'clim'] = 1
view3.by_channel_params['ch0', 'visible'] = True
view3.by_channel_params['ch1', 'visible'] = True
view3.by_channel_params['ch2', 'visible'] = True
view3.by_channel_params['ch4', 'visible'] = True
view3.by_channel_params['ch5', 'visible'] = True
view3.by_channel_params['ch6', 'visible'] = True
view3.by_channel_params['ch3', 'visible'] = False


win.add_view(view1)
win.add_view(view3)

#Run

win.show()
app.exec_()

debug True
QT_MODE PyQt5
refresh duration for  0.0 s
refresh duration for  0.0 s
save_all_settings


0

In [13]:
%gui qt
app = mkQApp()

sample_rate = 1000.
t_start = 0.

#Create the main window that can contain several viewers
win = MainViewer()
view1 = TraceViewer.from_numpy(All, sample_rate, t_start, 'Signals')
#add them to mainwindow

view1.params['display_labels'] = True
view1.params['scale_mode'] = 'same_for_all'
view1.auto_scale()

win.add_view(view1)


#Run
win.show()
